In [7]:
import s3manager
import pandas as pd
from pandas import DataFrame, Series

In [11]:
df_users = pd.read_pickle('df_users.pickle')
df_comments_meta = pd.read_pickle('df_comments_meta.pickle')
df_clubs = pd.read_pickle('club_names.pickle')
s_rubric_names = pd.read_pickle('rubric_names.pickle')
s_club_names = pd.read_pickle('club_names.pickle')

In [5]:
objects = []
for n in range(len(s_rubric_names)):
    key = 'rubrics/unique_primary/{}.json'.format(n+1)
    js = s3manager._s3_get_object(key)
    objects.append(js)

In [8]:
datas = []
for obj in objects:
    group = obj['group']
    rubric_id = obj['num']
    for comment in obj['comments']:
        club_id, post_id, dummy = comment[0][0].split('_')
        data = Series(dict(zip(['comment_id', 'user_id', 'rubric_id', 'group', 'text', 'characteristic', 'note', 'club_id', 'post_id'],
                        [comment[0][0], comment[1], rubric_id, group, comment[0][1], None, None, club_id, post_id])))
        datas.append(data)

In [ ]:
df = pd.concat([DataFrame([x]) for x in datas], ignore_index=True)
df.club_id = df.club_id.astype('int')
df = df.join(df_clubs, on='club_id')
df = df.join(s_rubric_names, on='rubric_id')

In [ ]:
df = df.merge(df_users, on='user_id')
df = df.join(df_comments_meta, on='comment_id')
df_indexed = df.set_index('comment_id')
df_indexed.bdate = pd.to_datetime(df_indexed.bdate)
df_indexed.to_pickle('df_indexed.pickle')